In [5]:
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract
import pandas as pd


In [6]:
text = pytesseract.image_to_string('test2.png')

text

'Capacité vitale Pleth. [Ll]\nVol. Residuel Pleth. [l]\nCap. Pulm. Totale Pleth. [l]\nCRF pléthysmo. [L]\nVR % CPT pleth. [%]\n\nConductances spécifiques [1/(KPA*S)]\n\nCV Forcée Expi [l]\nVEMS tu\nVEMS % CV MAX [%]\n\nDEMM 25/75 [l/s]\n\nThéo.\n\n2.76\n1.79\n4.70\n\n2.62\n38.34\n\n1.04\n2.67\n\n2.26\n78.3\n2gy/\n\nLIN\n\n2.07\nLal\n3.72\n\n1.80\n28.75\n\n1.04\n1.96\n\n1.63\n67.6\n1.57\n\n241\n3.83\n6.24\n\n5.01\n61.36\n\n0.35\n2.18\n\n1.12\n46.4\n0.48\n\n% Théo.\n\n87\n214\n133\n\n191\n160\n\n34\n82\n\n50\n9)\n16\n'

## Simple processing

In [26]:
# Splitting the text into sections for LIN, Basal, and % Théo
sections = text.split('\n')
sections

['Capacité vitale Pleth. [Ll]',
 'Vol. Residuel Pleth. [l]',
 'Cap. Pulm. Totale Pleth. [l]',
 'CRF pléthysmo. [L]',
 'VR % CPT pleth. [%]',
 '',
 'Conductances spécifiques [1/(KPA*S)]',
 '',
 'CV Forcée Expi [l]',
 'VEMS tu',
 'VEMS % CV MAX [%]',
 '',
 'DEMM 25/75 [l/s]',
 '',
 'Théo.',
 '',
 '2.76',
 '1.79',
 '4.70',
 '',
 '2.62',
 '38.34',
 '',
 '1.04',
 '2.67',
 '',
 '2.26',
 '78.3',
 '2gy/',
 '',
 'LIN',
 '',
 '2.07',
 'Lal',
 '3.72',
 '',
 '1.80',
 '28.75',
 '',
 '1.04',
 '1.96',
 '',
 '1.63',
 '67.6',
 '1.57',
 '',
 '241',
 '3.83',
 '6.24',
 '',
 '5.01',
 '61.36',
 '',
 '0.35',
 '2.18',
 '',
 '1.12',
 '46.4',
 '0.48',
 '',
 '% Théo.',
 '',
 '87',
 '214',
 '133',
 '',
 '191',
 '160',
 '',
 '34',
 '82',
 '',
 '50',
 '9)',
 '16',
 '']

In [27]:
parameters = structured_list[:13]
theo_values = structured_list[15:28:3]
lin_values = structured_list[30:43:3]
percent_theo_values = structured_list[45:58:3]


['Capacité vitale Pleth. [Ll]',
 'Cap. Pulm. Totale Pleth. [l]',
 'CRF pléthysmo. [L]']

In [ ]:

lin_values = sections[4].split("\n")
basal_values = sections[5].split("\n")
theo_values = sections[7].split("\n")


In [17]:
sections[4]

'Théo.'

In [ ]:

# Removing any extraneous values or inconsistencies in the data
while 'Théo.' in lin_values: lin_values.remove('Théo.')
while 'LIN' in basal_values: basal_values.remove('LIN')
while '% Théo.' in theo_values: theo_values.remove('% Théo.')

# Constructing the dataframe
df = pd.DataFrame({
    'Parameters': parameters_section,
    'LIN': lin_values,
    'Basal': basal_values,
    '% Théo': theo_values
})

df


## More extensive processing